<a href="https://www.kaggle.com/code/mmellinger66/ps3e6-paris-housing-models?scriptVersionId=119774256" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

 <div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Playground Season 3: Episode 6 - Paris Housing Prices</h1>
</div>

## Problem Type

Regression

## Evaluation Metric


<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Import Libraries</h1>
</div>

In [1]:
from typing import List, Set, Dict, Tuple, Optional

import os
import time
from pathlib import Path
import glob
import gc

import pandas as pd
import numpy as np

from sklearn import impute
from sklearn import metrics
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import svm
from sklearn import cluster
from sklearn import model_selection
from sklearn import ensemble
from sklearn import datasets

import xgboost as xgb
import catboost as cb
import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances

from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

# Visualization Libraries
import matplotlib as mpl
import matplotlib.pylab as plt
import seaborn as sns
import missingno as msno
from folium import Map
from folium.plugins import HeatMap
from IPython.display import display_html, display_markdown, display_latex
from colorama import Fore, Style

import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_rows", 999)
pd.set_option("display.precision", 5)

<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Configuration</h1>
</div>

In [2]:
TARGET="price"
ID="id"

In [3]:
class Config:
    path:str = "../input/playground-series-s3e6/"
    gpu:bool = True
    optimize:bool = True
    n_optuna_trials:int = 30
    fast_render:bool = False
    calc_probability:bool = False
    debug:bool = False
    seed:int = 42
    N_ESTIMATORS:int = 2000  # 100, 300, 1000, 2000, 5000, 15_000, 20_000 GBDT
    GPU_N_ESTIMATORS:int = 2000 # Want models to run fast during dev
    N_FOLDS:int = 5

In [4]:
class clr:
    S = Style.BRIGHT + Fore.LIGHTRED_EX
    E = Style.RESET_ALL

<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Library</h1>
</div>

In [5]:
def read_data(path: str, analyze:bool=True) -> (pd.DataFrame, pd.DataFrame, pd.DataFrame):
    data_dir = Path(path)

    train = pd.read_csv(data_dir / "train.csv")
    test = pd.read_csv(data_dir / "test.csv")
    submission_df = pd.read_csv(data_dir / "sample_submission.csv")

    if analyze:
        print(clr.S + "=== Shape of Data ==="+clr.E)
        print(f" train data: Rows={train.shape[0]}, Columns={train.shape[1]}")
        print(f" test data : Rows={test.shape[0]}, Columns={test.shape[1]}")

        print(clr.S + "\n=== Train Data: First 5 Rows ===\n"+clr.E)
        display(train.head())
        print(f"\n{clr.S}=== Train Column Names ==={clr.E}\n")
        display(train.columns)
        print(f"\n{clr.S}=== Features/Explanatory Variables ==={clr.E}\n")
        eval_features(train)
        print(f"\n{clr.S}=== Skewness ==={clr.E}\n")
        check_skew(train)
    return train, test, submission_df

def create_submission(model_name: str, target, preds, seed:int=42, nfolds:int=5) -> pd.DataFrame:
    sample_submission[target] = preds #.astype(int)

    if len(model_name) > 0:
        fname = f"submission_{model_name}_k{nfolds}_s{seed}.csv"
    else:
        fname = "submission.csv"

    sample_submission.to_csv(fname, index=False)

    return sample_submission

def show_classification_scores(ground_truth:List[int], yhat:List[int]) -> None:
    accuracy = metrics.accuracy_score(ground_truth, yhat)
    precision = metrics.precision_score(ground_truth, yhat)
    recall = metrics.recall_score(ground_truth, yhat)
    roc = metrics.roc_auc_score(ground_truth, yhat)
    f1 = metrics.f1_score(ground_truth, yhat)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"ROC: {roc:.4f}")
    print(f"f1: {f1:.4f}")
    

def label_encoder(train:pd.DataFrame, test:pd.DataFrame, columns:List[str]) -> (pd.DataFrame, pd.DataFrame) :
    for col in columns:
        train[col] = train[col].astype(str)
        test[col] = test[col].astype(str)
        train[col] = preprocessing.LabelEncoder().fit_transform(train[col])
        test[col] = preprocessing.LabelEncoder().fit_transform(test[col])
    return train, test   

def create_strat_folds(df:pd.DataFrame, TARGET, n_folds:int=5, seed:int=42) -> pd.DataFrame:
    print(f"TARGET={TARGET}, n_folds={n_folds}, seed={seed}")
    df["fold"] = -1

    kf = model_selection.StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
    # kf = GroupKFold(n_splits=Config.N_FOLDS)
    for fold, (train_idx, valid_idx) in enumerate(kf.split(df, df[TARGET])):
        df.loc[valid_idx, "fold"] = fold

    # df.to_csv(f"train_fold{num_folds}.csv", index=False)
    return df


def create_folds(df:pd.DataFrame, n_folds:int=5, seed:int=42) -> pd.DataFrame:
    print(f"n_folds={n_folds}, seed={seed}")
    df["fold"] = -1

    kf = model_selection.KFold(n_splits=n_folds, shuffle=True, random_state=seed)

    for fold, (train_idx, valid_idx) in enumerate(kf.split(df)):
        df.loc[valid_idx, "fold"] = fold

    # df.to_csv(f"train_fold{num_folds}.csv", index=False)
    return df

def show_fold_scores(scores: List[float]) -> (float, float):
    cv_score = np.mean(scores)  # Used in filename
    std_dev = np.std(scores)
    print(
        f"Scores -> Adjusted: {np.mean(scores) - np.std(scores):.8f} , mean: {np.mean(scores):.8f}, std: {np.std(scores):.8f}"
    )
    return cv_score, std_dev


def feature_distribution_types(df:pd.DataFrame, display:bool=True) -> (List[str], List[str]):
    continuous_features = list(df.select_dtypes(include=['int64', 'float64', 'uint8']).columns)
    categorical_features = list(df.select_dtypes(include=['object', 'bool']).columns)
    if display:
        print(f"{clr.S}Continuous Features={continuous_features}{clr.E}\n")
        print(f"{clr.S}Categorical Features={categorical_features}{clr.E}")
    return continuous_features, categorical_features   

def show_cardinality(df:pd.DataFrame, features:List[str]) -> None:
    print("=== Cardinality ===")
    print(df[features].nunique())

## === Model Support ===    

from scipy.stats import mode


def merge_test_predictions(final_test_predictions:List[float], calc_probability:bool=True) -> List[float]:

    if calc_probability:
        print("Mean")
        result = np.mean(np.column_stack(final_test_predictions), axis=1)
    else:
        print("Mode")
        mode_result = mode(np.column_stack(final_test_predictions), axis=1)
        result = mode_result[0].ravel()

    return result

def summary_statistics(X:pd.DataFrame, enhanced=True) -> None:
    desc = X.describe()
    if enhanced:
        desc.loc["var"] = X.var(numeric_only=True).tolist()
        desc.loc["skew"] = X.skew(numeric_only=True).tolist()
        desc.loc["kurt"] = X.kurtosis(numeric_only=True).tolist()

    with pd.option_context("display.precision", 2):
        style = desc.transpose().style.background_gradient(
            cmap="coolwarm"
        )  # .set_precision(4)
    display(style)
    
def show_missing_features(df:pd.DataFrame) -> None:
    missing_vals = df.isna().sum().sort_values(ascending=False)
    print(missing_vals[missing_vals > 0])


def show_duplicate_records(df:pd.DataFrame) -> None:
    dups = df.duplicated()
    print(dups.sum())


def eval_features(df:pd.DataFrame) -> (List[str], List[str], List[str]):
    ## Separate Categorical and Numerical Features
    categorical_features = list(
        df.select_dtypes(include=["category", "object"]).columns
    )
    continuous_features = list(df.select_dtypes(include=["number"]).columns)

    print(f"{clr.S}Continuous features:{clr.E} {continuous_features}")
    print(f"{clr.S}Categorical features:{clr.E} {categorical_features}")
    print("\n --- Cardinality of Categorical Features ---\n")

    for feature in categorical_features:
        cardinality = df[feature].nunique()
        if cardinality < 10:
            print(f"{clr.S}{feature}{clr.E}: cardinality={cardinality}, {df[feature].unique()}")
        else:
            print(f"{clr.S}{feature}{clr.E}: cardinality={cardinality}")
    all_features = categorical_features + continuous_features
    return all_features, categorical_features, continuous_features


def show_feature_importance(feature_importance_lst:List[str]) -> None:
    fis_df = pd.concat(feature_importance_lst, axis=1)

    fis_df.sort_values("0_importance", ascending=True).head(40).plot(
        kind="barh", figsize=(12, 12), title="Feature Importance Across Folds"
    )
    plt.show()


def show_feature_target_crosstab(df:pd.DataFrame, feature_lst:List[str], target:str) -> None:
    for feature in feature_lst:
        print(f"\n=== {feature} vs {target} ===\n")
        display(
            pd.crosstab(df[feature], df[target], margins=True)
        )  # display keeps bold formatting


def show_cardinality(df:pd.DataFrame, features:List[str]) -> None:
    print(f"{clr.S}=== Cardinality ==={clr.E}")
    print(df[features].nunique())


def show_unique_features(df:pd.DataFrame, features:List[str]) -> None:
    for col in features:
        print(col, sorted(df[col].dropna().unique()))


def feature_distribution_types(df:pd.DataFrame, display:bool=True) -> (List[str], List[str]):
    continuous_features = list(
        df.select_dtypes(include=["int64", "float64", "uint8"]).columns
    )
    categorical_features = list(df.select_dtypes(include=["object", "bool"]).columns)
    if display:
        print(f"{clr.S}Continuous Features={clr.E}{continuous_features}\n")
        print(f"{clr.S}Categorical Features={clr.E}{categorical_features}")
    return continuous_features, categorical_features


def describe(X:pd.DataFrame) -> None:
    "Deprecated: Use summary_statistics()"
    desc = X.describe()
    desc.loc['var'] = X.var(numeric_only=True).tolist()
    desc.loc['skew'] = X.skew(numeric_only=True).tolist()
    desc.loc['kurt'] = X.kurtosis(numeric_only=True).tolist()

    with pd.option_context('display.precision', 2):
        style = desc.transpose().style.background_gradient(cmap='coolwarm') #.set_precision(4)
    display(style)
  

def check_skew(df:pd.DataFrame) -> None:
    skew = df.skew(skipna=True,numeric_only=True).sort_values(ascending=False)
    print(skew)
    
def gpu_ify_lgbm(lgbm_dict):
    lgbm_dict["device"] = "gpu"
    lgbm_dict["boosting_type"] = "gbdt"
    lgbm_dict["gpu_platform_id"] = 0
    lgbm_dict["gpu_device_id"] = 0
    return lgbm_dict

def gpu_ify_cb(params):
    params["task_type"] = "GPU"
    return params    


<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Optuna Hyperparameter Optimization Library</h1>
</div>

In [6]:
def objective_xgb(trial, X_train, X_valid, y_train, y_valid):

    xgb_params = {
        #         "objective": trial.suggest_categorical("objective", ["multi:softmax"]),
        #         "eval_metric": "mlogloss",
        #         "objective": "multi:softmax",
        "eval_metric": "rmse",  # auc, rmse, mae
        "objective": "reg:squarederror",
        #         "enable_categorical": trial.suggest_categorical("use_label_encoder", [True]),
        "use_label_encoder": trial.suggest_categorical("use_label_encoder", [False]),
        "n_estimators": trial.suggest_int("n_estimators", 1000, 5000, 100),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-2, 0.25),
        "subsample": trial.suggest_float("subsample", 0.1, 1, step=0.01),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1, step=0.01),
        "max_depth": trial.suggest_int("max_depth", 1, 20),  # 10
        "gamma": trial.suggest_float("gamma", 0, 100, step=0.1),
        "booster": trial.suggest_categorical("booster", ["gbtree"]),
        "tree_method": trial.suggest_categorical(
            "tree_method", ["gpu_hist"]
        ),  # hist, gpu_hist
        "predictor": "gpu_predictor",
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-8, 100),
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-8, 100),
        "random_state": trial.suggest_categorical("random_state", [42]),
        "n_jobs": trial.suggest_categorical("n_jobs", [4]),
        "min_child_weight": trial.suggest_loguniform("min_child_weight", 1e-1, 1e3),
        # "min_child_weight": trial.suggest_categorical("min_child_weight", [256]),
    }

    # Model loading and training
    model = xgb.XGBRegressor(**xgb_params)
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        early_stopping_rounds=5000,
        verbose=0,
    )

    print(f"Number of boosting rounds: {model.best_iteration}")
    #     oof = model.predict_proba(X_valid)[:, 1] # Probability
    oof = model.predict(X_valid)  # Classification: 0,1

    return metrics.mean_squared_error(y_valid, oof, squared=False)


def objective_lgbm(trial, X_train, X_valid, y_train, y_valid):

    lgbm_params = {
        "objective": trial.suggest_categorical("objective", ["mae", "rmse"]),
        #         "n_estimators": trial.suggest_categorical("n_estimators", [1_000]),
        #         "n_estimators": trial.suggest_categorical("n_estimators", [5000]),
        "n_estimators": trial.suggest_int("n_estimators", 700, 5000),
        "importance_type": "gain",
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-8, 10.0),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-8, 10.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1, step=0.01),
        "num_leaves": trial.suggest_int("num_leaves", 2, 1000),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.1, 1.0),
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.1, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 0, 15),
        "min_child_samples": trial.suggest_int("min_child_samples", 1, 300),
        "subsample": trial.suggest_float("subsample", 0.1, 1, step=0.01),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-2, 0.25),
        "max_depth": trial.suggest_int("max_depth", 1, 100),
        "random_state": trial.suggest_categorical("random_state", [42]),
        "n_jobs": trial.suggest_categorical("n_jobs", [4]),
        #         'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-1, 1e3),
        # "min_child_weight": trial.suggest_categorical("min_child_weight", [256]),
    }

    # Model loading and training
    model = lgb.LGBMRegressor(**lgbm_params)
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_metric="mae",
        callbacks=[
            lgb.log_evaluation(500),
            lgb.early_stopping(500, False, True),
        ],
    )

    #     print(f"Number of boosting rounds: {model.best_iteration}")
    oof = model.predict(X_valid)

    return metrics.mean_squared_error(y_valid, oof, squared=False)
#     return metrics.mean_absolute_error(y_valid, oof)


def objective_clf_lgbm(trial, X_train, X_valid, y_train, y_valid):

    params = {
        "boosting_type": "gbdt",
        # "objective": trial.suggest_categorical("objective", ["mae", "rmse"]),
        #         "objective": trial.suggest_categorical("objective", ["multi:softprob"]),
        #         "n_estimators": trial.suggest_categorical("n_estimators", [1_000]),
        #         "n_estimators": trial.suggest_categorical("n_estimators", [5000]),
        "n_estimators": trial.suggest_int("n_estimators", 700, 1000),
        "importance_type": "gain",
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-8, 10.0),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-8, 10.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1, step=0.01),
        "num_leaves": trial.suggest_int("num_leaves", 2, 1000),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.1, 1.0),
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.1, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 0, 15),
        "min_child_samples": trial.suggest_int("min_child_samples", 1, 300),
        "subsample": trial.suggest_float("subsample", 0.1, 1, step=0.01),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-2, 0.25),
        "max_depth": trial.suggest_int("max_depth", 1, 100),
        "random_state": trial.suggest_categorical("random_state", [42]),
        "n_jobs": trial.suggest_categorical("n_jobs", [4]),
        #         'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-1, 1e3),
        # "min_child_weight": trial.suggest_categorical("min_child_weight", [256]),
    }
    if Config.gpu:
        params["device_type"] = "gpu"

    # Model loading and training
    model = lgb.LGBMClassifier(**params)
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        # eval_metric="mae",
        callbacks=[
            lgb.log_evaluation(500),
            lgb.early_stopping(500, False, True),
        ],
    )

    #     print(f"Number of boosting rounds: {model.best_iteration}")
    oof = model.predict(X_valid)

    #     return accuracy_score(y_valid, oof)
    return metrics.roc_auc_score(y_valid, oof)


def objective_cb(trial, X_train, X_valid, y_train, y_valid):

    cb_params = {
        "iterations": 100,
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.1, 1.0),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1, 100),
        "bagging_temperature": trial.suggest_loguniform(
            "bagging_temperature", 0.1, 20.0
        ),
        "random_strength": trial.suggest_float("random_strength", 1.0, 2.0),
        "depth": trial.suggest_int("depth", 1, 10),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 300),
          "use_best_model": True,
#         "task_type": "GPU",
        "random_seed": 42,
    }

    # Model loading and training
    #  model = CatBoostClassifier(**cb_params)
    model = cb.CatBoostRegressor(**cb_params)

    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        # eval_metric="accuracy",
        early_stopping_rounds=500,
        verbose=False,
    )

#     print(f"Number of boosting rounds: {model.best_iteration}")
    # oof = model.predict_proba(X_valid)[:, 1]
    oof = model.predict(X_valid)  # Classification
    return metrics.mean_squared_error(y_valid, oof, squared=False)
#     return metrics.mean_absolute_error(y_valid, oof)
# 
#     return accuracy_score(y_valid, oof)

def objective_clf_cb(trial, X_train, X_valid, y_train, y_valid):

    cb_params = {
        "iterations": 10,  # 1000
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.1, 1.0),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1, 100),
        "bagging_temperature": trial.suggest_loguniform(
            "bagging_temperature", 0.1, 20.0
        ),
        "random_strength": trial.suggest_float("random_strength", 1.0, 2.0),
        "depth": trial.suggest_int("depth", 1, 10),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 300),
        "use_best_model": True,
#             "task_type": "GPU",
        "random_seed": 42,
    }

    # Model loading and training
    model = cb.CatBoostClassifier(**cb_params)
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        # eval_metric="accuracy",
        early_stopping_rounds=500,
        verbose=False,
    )

    # print(f"Number of boosting rounds: {model.best_iteration}")
    # oof = model.predict_proba(X_valid)[:, 1]
    oof = model.predict(X_valid)  # Classification

    return metrics.accuracy_score(y_valid, oof)

<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Load Train/Test Data and Analyze</h1>
</div>

## Load the following files

 - train.csv - Data used to build our machine learning model
 - test.csv - Data used to build our machine learning model. Does not contain the target variable
 - sample_submission.csv - A file in the proper format to submit test predictions

In [7]:
%%time
train, test, sample_submission = read_data(Config.path, analyze=True)                                

=== Shape of Data ===
 train data: Rows=22730, Columns=18
 test data : Rows=15154, Columns=17

=== Train Data: First 5 Rows ===



,id,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,0,34291,24,1,0,47,35693,2,1,2000,0,1,8,5196,369,0,3,3436795.2
1,1,95145,60,0,1,60,34773,1,4,2000,0,1,729,4496,277,0,6,9519958.0
2,2,92661,45,1,1,62,45457,4,8,2020,1,1,7473,8953,245,1,9,9276448.1
3,3,97184,99,0,0,59,15113,1,1,2000,0,1,6424,8522,256,1,9,9725732.2
4,4,61752,100,0,0,57,64245,8,4,2018,1,0,7151,2786,863,0,7,6181908.8



=== Train Column Names ===



Index(['id', 'squareMeters', 'numberOfRooms', 'hasYard', 'hasPool', 'floors',
       'cityCode', 'cityPartRange', 'numPrevOwners', 'made', 'isNewBuilt',
       'hasStormProtector', 'basement', 'attic', 'garage', 'hasStorageRoom',
       'hasGuestRoom', 'price'],
      dtype='object')


=== Features/Explanatory Variables ===

Continuous features: ['id', 'squareMeters', 'numberOfRooms', 'hasYard', 'hasPool', 'floors', 'cityCode', 'cityPartRange', 'numPrevOwners', 'made', 'isNewBuilt', 'hasStormProtector', 'basement', 'attic', 'garage', 'hasStorageRoom', 'hasGuestRoom', 'price']
Categorical features: []

 --- Cardinality of Categorical Features ---


=== Skewness ===

floors               85.12233
squareMeters         79.25331
made                 66.93441
basement              3.33664
attic                 2.80996
garage                1.37876
cityCode              0.24580
hasPool               0.18986
hasStormProtector     0.16049
hasStorageRoom        0.15374
price                 0.13140
isNewBuilt            0.12838
numberOfRooms         0.11508
hasYard               0.09656
id                    0.00000
numPrevOwners        -0.07711
cityPartRange        -0.09306
hasGuestRoom         -0.11777
dtype: float64
CPU times: user 65.7 ms, sys: 20 ms, total: 85.7 ms
Wall 

In [8]:
train.head()

,id,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,0,34291,24,1,0,47,35693,2,1,2000,0,1,8,5196,369,0,3,3436795.2
1,1,95145,60,0,1,60,34773,1,4,2000,0,1,729,4496,277,0,6,9519958.0
2,2,92661,45,1,1,62,45457,4,8,2020,1,1,7473,8953,245,1,9,9276448.1
3,3,97184,99,0,0,59,15113,1,1,2000,0,1,6424,8522,256,1,9,9725732.2
4,4,61752,100,0,0,57,64245,8,4,2018,1,0,7151,2786,863,0,7,6181908.8


In [9]:
original = pd.read_csv("../input/paris-housing-price-prediction/ParisHousing.csv")

original.head()

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,75523,3,0,1,63,9373,3,8,2005,0,1,4313,9005,956,0,7,7559081.5
1,80771,39,1,1,98,39381,8,6,2015,1,0,3653,2436,128,1,2,8085989.5
2,55712,58,0,1,19,34457,6,8,2021,0,0,2937,8852,135,1,9,5574642.1
3,32316,47,0,0,6,27939,10,4,2012,0,1,659,7141,359,0,3,3232561.2
4,70429,19,1,1,90,38045,3,7,1990,1,0,8435,2429,292,1,4,7055052.0


In [10]:
original = original.reset_index()
original['id'] = original['index'] + 1000000
original = original.drop(columns = ['index'])
original.head()

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price,id
0,75523,3,0,1,63,9373,3,8,2005,0,1,4313,9005,956,0,7,7559081.5,1000000
1,80771,39,1,1,98,39381,8,6,2015,1,0,3653,2436,128,1,2,8085989.5,1000001
2,55712,58,0,1,19,34457,6,8,2021,0,0,2937,8852,135,1,9,5574642.1,1000002
3,32316,47,0,0,6,27939,10,4,2012,0,1,659,7141,359,0,3,3232561.2,1000003
4,70429,19,1,1,90,38045,3,7,1990,1,0,8435,2429,292,1,4,7055052.0,1000004


In [11]:
train['is_original']    = 0
test['is_original']     = 0
original['is_original'] = 1
combined = pd.concat([train, original], ignore_index=True)
train = combined

In [12]:
combined.head()

,id,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price,is_original
0,0,34291,24,1,0,47,35693,2,1,2000,0,1,8,5196,369,0,3,3436795.2,0
1,1,95145,60,0,1,60,34773,1,4,2000,0,1,729,4496,277,0,6,9519958.0,0
2,2,92661,45,1,1,62,45457,4,8,2020,1,1,7473,8953,245,1,9,9276448.1,0
3,3,97184,99,0,0,59,15113,1,1,2000,0,1,6424,8522,256,1,9,9725732.2,0
4,4,61752,100,0,0,57,64245,8,4,2018,1,0,7151,2786,863,0,7,6181908.8,0


In [13]:
summary_statistics(train.drop(columns=[ID], axis=1), enhanced=True)

,count,mean,std,min,25%,50%,75%,max,var,skew,kurt
squareMeters,32730.00,47589.55,44252.70,89.00,21581.00,46132.00,72545.00,6071330.00,1958301742.77,77.09,10489.20
numberOfRooms,32730.00,48.89,28.42,1.00,25.00,48.00,75.00,100.00,807.94,0.09,-1.16
hasYard,32730.00,0.49,0.50,0.00,0.00,0.00,1.00,1.00,0.25,0.06,-2.00
hasPool,32730.00,0.47,0.50,0.00,0.00,0.00,1.00,1.00,0.25,0.14,-1.98
floors,32730.00,48.21,42.92,1.00,25.00,46.00,72.00,6000.00,1842.06,81.51,11298.59
cityCode,32730.00,50078.47,29704.41,3.00,23446.00,50452.00,76229.00,491100.00,882351901.11,0.17,1.43
cityPartRange,32730.00,5.56,2.78,1.00,3.00,6.00,8.00,10.00,7.73,-0.06,-1.17
numPrevOwners,32730.00,5.59,2.76,1.00,3.00,6.00,8.00,10.00,7.61,-0.05,-1.14
made,32730.00,2007.24,99.16,1990.00,2000.00,2006.00,2014.00,10000.00,9833.46,80.00,6446.00
isNewBuilt,32730.00,0.48,0.50,0.00,0.00,0.00,1.00,1.00,0.25,0.09,-1.99


<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Feature Engineering</h1>
</div>

## Categorical/Numerical Variables

In [14]:
train.drop(['cityCode'], axis=1, inplace=True)
test.drop(['cityCode'], axis=1, inplace=True)


In [15]:
cont_features, cat_features = feature_distribution_types(train, display=True)
show_cardinality(train, cat_features)

cont_features.remove(TARGET)
cont_features.remove(ID)
FEATURES = cont_features + cat_features
FEATURES

Continuous Features=['id', 'squareMeters', 'numberOfRooms', 'hasYard', 'hasPool', 'floors', 'cityPartRange', 'numPrevOwners', 'made', 'isNewBuilt', 'hasStormProtector', 'basement', 'attic', 'garage', 'hasStorageRoom', 'hasGuestRoom', 'price', 'is_original']

Categorical Features=[]
=== Cardinality ===
Series([], dtype: float64)


['squareMeters',
 'numberOfRooms',
 'hasYard',
 'hasPool',
 'floors',
 'cityPartRange',
 'numPrevOwners',
 'made',
 'isNewBuilt',
 'hasStormProtector',
 'basement',
 'attic',
 'garage',
 'hasStorageRoom',
 'hasGuestRoom',
 'is_original']

In [16]:
excluded_features = [TARGET, ID, "fold"]

In [17]:
cont_features, cat_features = feature_distribution_types(train, display=True)
show_cardinality(train, cat_features)

cont_features = [feature for feature in cont_features if feature not in excluded_features]
cat_features = [feature for feature in cat_features if feature not in excluded_features]

FEATURES = cont_features + cat_features
FEATURES

Continuous Features=['id', 'squareMeters', 'numberOfRooms', 'hasYard', 'hasPool', 'floors', 'cityPartRange', 'numPrevOwners', 'made', 'isNewBuilt', 'hasStormProtector', 'basement', 'attic', 'garage', 'hasStorageRoom', 'hasGuestRoom', 'price', 'is_original']

Categorical Features=[]
=== Cardinality ===
Series([], dtype: float64)


['squareMeters',
 'numberOfRooms',
 'hasYard',
 'hasPool',
 'floors',
 'cityPartRange',
 'numPrevOwners',
 'made',
 'isNewBuilt',
 'hasStormProtector',
 'basement',
 'attic',
 'garage',
 'hasStorageRoom',
 'hasGuestRoom',
 'is_original']

<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Optuna Hyperparameter Optimization</h1>
</div>

In [18]:
%%time

if Config.optimize:
    y = train[TARGET]
    X = train[FEATURES].copy()

    X_test = test[FEATURES].copy()
    X_train, X_valid, y_train, y_valid = model_selection.train_test_split(
        X, y, test_size=0.2, random_state=Config.seed
    )

# === XGB ===

time_limit = 3600 * 3
best_xgb_params = {}

if Config.optimize:
    study = optuna.create_study(direction="maximize")
    study.optimize(
        lambda trial: objective_xgb(trial, X_train, X_valid, y_train, y_valid),
        n_trials=Config.n_optuna_trials,
        # timeout=time_limit,  # this or n_trials
    )

if Config.optimize:
    print("Number of finished trials:", len(study.trials))
    print("Best trial parameters:", study.best_trial.params)
    print("Best score:", study.best_value)
    best_xgb_params = study.best_trial.params

## === LGBM ===

time_limit = 3600 * 3
best_lgbm_params = {}

if Config.optimize:
    study = optuna.create_study(direction="maximize")
    study.optimize(
        lambda trial: objective_lgbm(trial, X_train, X_valid, y_train, y_valid),
        n_trials=Config.n_optuna_trials,
        # timeout=time_limit,  # this or n_trials
    )

if Config.optimize:
    print("Number of finished trials:", len(study.trials))
    print("Best trial parameters:", study.best_trial.params)
    print("Best score:", study.best_value)
    best_lgbm_params = study.best_trial.params

## === CatBoost

time_limit = 3600 * 3
best_cb_params = {}

if Config.optimize:
    study = optuna.create_study(direction="maximize")
    study.optimize(
        lambda trial: objective_cb(trial, X_train, X_valid, y_train, y_valid),
        n_trials=Config.n_optuna_trials,
        # timeout=time_limit,  # this or n_trials
    )

if Config.optimize:
    print("Number of finished trials:", len(study.trials))
    print("Best trial parameters:", study.best_trial.params)
    print("Best score:", study.best_value)
    best_cb_params = study.best_trial.params

[I 2023-02-20 17:56:09,804] A new study created in memory with name: no-name-16ce9555-c8a0-4132-abf0-4f380086f776
[I 2023-02-20 17:57:08,218] Trial 0 finished with value: 215330.73989893027 and parameters: {'use_label_encoder': False, 'n_estimators': 3200, 'learning_rate': 0.07502172100141696, 'subsample': 0.36, 'colsample_bytree': 0.8500000000000001, 'max_depth': 12, 'gamma': 94.5, 'booster': 'gbtree', 'tree_method': 'gpu_hist', 'reg_lambda': 0.10631323726463023, 'reg_alpha': 4.267395112269372, 'random_state': 42, 'n_jobs': 4, 'min_child_weight': 9.321034714762156}. Best is trial 0 with value: 215330.73989893027.


Number of boosting rounds: 324


[I 2023-02-20 17:57:14,278] Trial 1 finished with value: 184553.72935743703 and parameters: {'use_label_encoder': False, 'n_estimators': 4700, 'learning_rate': 0.018802691772853444, 'subsample': 0.45000000000000007, 'colsample_bytree': 0.05, 'max_depth': 3, 'gamma': 14.0, 'booster': 'gbtree', 'tree_method': 'gpu_hist', 'reg_lambda': 0.0006052691569581149, 'reg_alpha': 6.472908437565326e-08, 'random_state': 42, 'n_jobs': 4, 'min_child_weight': 0.1129203944253971}. Best is trial 0 with value: 215330.73989893027.


Number of boosting rounds: 4699


[I 2023-02-20 17:59:41,732] Trial 2 finished with value: 675995.5290580511 and parameters: {'use_label_encoder': False, 'n_estimators': 2600, 'learning_rate': 0.1571476590255056, 'subsample': 0.30000000000000004, 'colsample_bytree': 0.7500000000000001, 'max_depth': 20, 'gamma': 11.5, 'booster': 'gbtree', 'tree_method': 'gpu_hist', 'reg_lambda': 0.054906656486128, 'reg_alpha': 2.135308819258508e-06, 'random_state': 42, 'n_jobs': 4, 'min_child_weight': 1.4106216621070111}. Best is trial 2 with value: 675995.5290580511.


...

[LightGBM] [Warning] bagging_fraction is set=0.9482635290957784, subsample=0.29000000000000004 will be ignored. Current value: bagging_fraction=0.9482635290957784
[LightGBM] [Warning] feature_fraction is set=0.4321217137609482, colsample_bytree=0.3 will be ignored. Current value: feature_fraction=0.4321217137609482
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
Training until validation scores don't improve for 500 rounds
[500]	training's l1: 27363.9	training's rmse: 64923.2	valid_1's l1: 234829	valid_1's rmse: 326730
[1000]	training's l1: 7617.73	training's rmse: 30902.7	valid_1's l1: 235224	valid_1's rmse: 326782
Early stopping, best iteration is:
[656]	training's l1: 17347	training's rmse: 49470.9	valid_1's l1: 234758	valid_1's rmse: 326457


[I 2023-02-20 19:57:16,100] Trial 29 finished with value: 326457.49010323547 and parameters: {'objective': 'rmse', 'n_estimators': 1313, 'reg_alpha': 0.0006564666842286576, 'reg_lambda': 5.238298684302513e-07, 'colsample_bytree': 0.3, 'num_leaves': 965, 'feature_fraction': 0.4321217137609482, 'bagging_fraction': 0.9482635290957784, 'bagging_freq': 2, 'min_child_samples': 64, 'subsample': 0.29000000000000004, 'learning_rate': 0.1297702792026085, 'max_depth': 54, 'random_state': 42, 'n_jobs': 4}. Best is trial 23 with value: 509363.70658724464.
[I 2023-02-20 19:57:16,113] A new study created in memory with name: no-name-20e4ec17-0a94-493d-90e2-a6a3137cf00b


Number of finished trials: 30
Best trial parameters: {'objective': 'rmse', 'n_estimators': 2700, 'reg_alpha': 0.000599103298257731, 'reg_lambda': 3.0040174328056107e-07, 'colsample_bytree': 0.5, 'num_leaves': 806, 'feature_fraction': 0.4151251951012078, 'bagging_fraction': 0.8556709748326068, 'bagging_freq': 2, 'min_child_samples': 24, 'subsample': 0.22, 'learning_rate': 0.08346471947257707, 'max_depth': 90, 'random_state': 42, 'n_jobs': 4}
Best score: 509363.70658724464


[I 2023-02-20 19:57:17,027] Trial 0 finished with value: 171094.22462679425 and parameters: {'learning_rate': 0.1495954440852017, 'l2_leaf_reg': 5.159377510044407, 'bagging_temperature': 0.1980944390067433, 'random_strength': 1.3386686714307992, 'depth': 7, 'min_data_in_leaf': 59}. Best is trial 0 with value: 171094.22462679425.
[I 2023-02-20 19:57:17,503] Trial 1 finished with value: 182479.94886388283 and parameters: {'learning_rate': 0.102639358268506, 'l2_leaf_reg': 5.159112337281242, 'bagging_temperature': 3.2491106381007233, 'random_strength': 1.4828935750672931, 'depth': 4, 'min_data_in_leaf': 157}. Best is trial 1 with value: 182479.94886388283.
[I 2023-02-20 19:57:18,101] Trial 2 finished with value: 183571.03345247198 and parameters: {'learning_rate': 0.13444555448984127, 'l2_leaf_reg': 97.29713937917167, 'bagging_temperature': 10.180495132362122, 'random_strength': 1.1648084842435833, 'depth': 6, 'min_data_in_leaf': 124}. Best is trial 2 with value: 183571.03345247198.
[I 20

Number of finished trials: 30
Best trial parameters: {'learning_rate': 0.45743264601999495, 'l2_leaf_reg': 41.338946049390074, 'bagging_temperature': 0.3472567739474319, 'random_strength': 1.7332249677756242, 'depth': 1, 'min_data_in_leaf': 6}
Best score: 296435.6534212222
CPU times: user 2h 12min 35s, sys: 8min 14s, total: 2h 20min 50s
Wall time: 2h 1min 24s


<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Train Models with Cross Validation</h1>
</div>

In [19]:
train = create_folds(train, Config.N_FOLDS)
# train = create_strat_folds(train, TARGET, Config.N_FOLDS)

n_folds=5, seed=42


In [20]:
all_cv_scores = pd.DataFrame(
    {
        "Model": pd.Series(dtype="str"),
        "Score": pd.Series(dtype="float"),
        "StdDev": pd.Series(dtype="float"),
        "RunTime": pd.Series(dtype="float"),
    }
)

oof = train[[ID, TARGET, "fold"]].copy().reset_index(drop=True).copy()
oof.set_index(ID, inplace=True)
oof.head()

,price,fold
id,,
0,3436795.2,2
1,9519958.0,3
2,9276448.1,3
3,9725732.2,1
4,6181908.8,1


In [21]:
def show_tree_model_fi(model, features:List[str]) -> None:
    print("\n=== Model Feature Importance ===")
    for i in model.feature_importances_.argsort()[::-1]:
        print(features[i], model.feature_importances_[i]/model.feature_importances_.sum())

def save_oof_predictions(model_name:str, final_valid_predictions, oof:pd.DataFrame) -> pd.DataFrame:
    final_valid_predictions_df = process_valid_predictions(
        final_valid_predictions, ID, model_name
    )
    display(final_valid_predictions_df.head())
    oof[f"pred_{model_name}"] = final_valid_predictions_df[f"pred_{model_name}"]

    return oof

def save_test_predictions(model_name:str, final_test_predictions, submission_df:pd.DataFrame, result_field:str=TARGET) -> None:
    result = merge_test_predictions(final_test_predictions, Config.calc_probability)
    # result[:20]
    submission_df[f"target_{model_name}"] = result #.astype(int)
    #     submission_df.head(10)
    ss = submission_df[[ID, f"target_{model_name}"]].copy().reset_index(drop=True)
    ss.rename(columns={f"target_{model_name}": result_field}, inplace=True)
    ss.to_csv(
        f"submission_{model_name}.csv", index=False
    )  # Can submit the individual model
    print("=== Target Value Counts ===")
#     display(ss[TARGET].value_counts())
    ss.head(10)

def process_valid_predictions(final_valid_predictions, train_id, model_name:str) -> pd.DataFrame:
    model = f"pred_{model_name}"
    final_valid_predictions_df = pd.DataFrame.from_dict(
        final_valid_predictions, orient="index"
    ).reset_index()
    final_valid_predictions_df.columns = [train_id, model]
    final_valid_predictions_df.set_index(train_id, inplace=True)
    final_valid_predictions_df.sort_index(inplace=True)
    final_valid_predictions_df.to_csv(f"train_pred_{model_name}.csv", index=True)

    return final_valid_predictions_df

def add_score(score_df:pd.DataFrame, model_name:str, score:float, std:float):
    dict1 = {"Model": model_name, "Score": cv_score, "StdDev": std_dev}
    score_df = score_df.append(dict1, ignore_index=True)
    return score_df

In [22]:
def train_cv_model(
    df:pd.DataFrame,
    test:pd.DataFrame,
    get_model_fn,
    FEATURES:List[str],
    TARGET:str,
    calc_probability:bool,
    rowid,
    params,
    n_folds:int=5,
    seed:int=42,
):

    final_test_predictions = []
    final_valid_predictions = {}
    fold_scores = []  # Scores of Validation Set
    feature_importance_lst = []

    test = test[FEATURES].copy()

    for fold in range(n_folds):
        print(10 * "=", f"Fold {fold+1}/{n_folds}", 10 * "=")

        start_time = time.time()

        xtrain = df[df.fold != fold].reset_index(
            drop=True
        )  # Everything not in validation fold
        xvalid = df[df.fold == fold].reset_index(drop=True)
        xtest = test.copy()

        valid_ids = xvalid.id.values.tolist()  # Id's of everything in validation fold

        ytrain = xtrain[TARGET]
        yvalid = xvalid[TARGET]

        xtrain = xtrain[FEATURES]
        xvalid = xvalid[FEATURES]

        scaler = preprocessing.StandardScaler()
#         scaler = preprocessing.MinMaxScaler()
        xtrain = scaler.fit(xtrain).transform(xtrain)
        xvalid = scaler.transform(xvalid)
        xtest = scaler.transform(xtest)

        model = get_model_fn # ()

        model.fit(
            xtrain,
            ytrain,
        )
        if calc_probability:
            preds_valid = model.predict_proba(xvalid)[:, 1]
            test_preds = model.predict_proba(xtest)[:, 1]
        else:
            preds_valid = model.predict(xvalid)
            test_preds = model.predict(xtest)

        preds_valid_class = model.predict(xvalid)
        
        final_test_predictions.append(test_preds)
        final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))

#         fold_score = metrics.accuracy_score(yvalid, preds_valid_class)  # Validation Set Score
        fold_score = metrics.mean_absolute_error(
            yvalid, preds_valid
        ) 
#         fold_score = metrics.roc_auc_score(yvalid.values, preds_valid)  # Validation Set Score
#         show_classification_scores(yvalid.values, preds_valid_class)

#         fold_score = metrics.roc_auc_score(yvalid, preds_valid)  # Validation Set Score
#         fold_score = metrics.mean_squared_error(yvalid, preds_valid, squared=False)
        fold_scores.append(fold_score)
        #         importance_list.append(model.coef_.ravel())

        fi = []
        # Feature importance
#         fi = pd.DataFrame(
#             index=FEATURES,
#             data=model.coef_.ravel(),
#             columns=[f"{fold}_importance"],
#         )
        
        feature_importance_lst.append(fi)

        run_time = time.time() - start_time

        print(f"fold: {fold+1}, Score: {fold_score}, Run Time: {run_time:.2f}")

    return (
        model,
        feature_importance_lst,
        fold_scores,
        final_valid_predictions,
        final_test_predictions,
    )


def train_xgb_model(
    df:pd.DataFrame,
    test:pd.DataFrame,
    get_model_fn,
    FEATURES:List[str],
    TARGET:str,
    calc_probability:bool,
    rowid:str,
    params,
    n_folds:int=5,
    seed:int=42,
):

    print(params)
    final_test_predictions = []
    final_valid_predictions = {}
    fold_scores = []  # Scores of Validation Set
    feature_importance_lst = []

    test = test[FEATURES].copy()

    for fold in range(n_folds):
        print(10 * "=", f"Fold {fold+1}/{n_folds}", 10 * "=")

        start_time = time.time()

        xtrain = df[df.fold != fold].reset_index(
            drop=True
        )  # Everything not in validation fold
        xvalid = df[df.fold == fold].reset_index(drop=True)
        xtest = test.copy()

        valid_ids = xvalid.id.values.tolist()  # Id's of everything in validation fold

        ytrain = xtrain[TARGET]
        yvalid = xvalid[TARGET]

        xtrain = xtrain[FEATURES]
        xvalid = xvalid[FEATURES]

        model = get_model_fn # (params)

        model.fit(
            xtrain,
            ytrain,
            eval_set=[(xvalid, yvalid)],
            #             eval_metric="acc",  # auc
            verbose=0,
            #             early_stopping_rounds=3000,
            #             callbacks=[
            #                 xgb.log_evaluation(0),
            #                 xgb.early_stopping(500, False, True),
            #             ],
        )

        if calc_probability:
            preds_valid = model.predict_proba(xvalid)[:, 1]
            test_preds = model.predict_proba(xtest)[:, 1]
        else:
            preds_valid = model.predict(xvalid)
            test_preds = model.predict(xtest)

        preds_valid_class = model.predict(xvalid)
        
        final_test_predictions.append(test_preds)
        if Config.debug:
            print(f"GT Type: {type(yvalid.values)}")
            print(f"Preds Type: {type(preds_valid_class)}")
            print(f"         GT:{yvalid.values[:20]}")
            print(f"Preds Class:{preds_valid_class[:20]}")
            print(f"Preds Prob:{preds_valid[:20]}")
        final_valid_predictions.update(dict(zip(valid_ids, preds_valid_class)))

#         fold_score = metrics.cohen_kappa_score(yvalid,  preds_valid_class, weights = "quadratic")
#         fold_score = metrics.roc_auc_score(yvalid.values, preds_valid)  # Validation Set Score
#         show_classification_scores(yvalid.values, preds_valid_class)
        fold_score = metrics.mean_absolute_error(
            yvalid, preds_valid
        )  # Validation Set Score
#         fold_score = metrics.mean_squared_error(yvalid, preds_valid, squared=False)
        fold_scores.append(fold_score)

        # Feature importance
        fi = pd.DataFrame(
            index=FEATURES,
            data=model.feature_importances_,
            columns=[f"{fold}_importance"],
        )
        feature_importance_lst.append(fi)

        run_time = time.time() - start_time

        print(f"fold: {fold+1}, Score: {fold_score}, Run Time: {run_time:.2f}")

    return (
        model,
        feature_importance_lst,
        fold_scores,
        final_valid_predictions,
        final_test_predictions,
    )        

In [23]:
def run_linear_model(model_dict, model_name:str, features:List[str], oof:pd.DataFrame) -> (float, float, pd.DataFrame):
    (
        model,
        feature_importance_lst,
        fold_scores,
        final_valid_predictions,
        final_test_predictions,
    ) = train_cv_model(
        train,
        test,
        model_dict[model_name],
        features,
        TARGET,
        False, #Config.calc_probability,
        ID,
        {},
        Config.N_FOLDS,
        Config.seed,
    )

    cv_score, std_dev = show_fold_scores(fold_scores)

    oof = save_oof_predictions(model_name, final_valid_predictions, oof)
    oof.head()
    save_test_predictions(model_name, final_test_predictions, sample_submission, TARGET)

    return cv_score, std_dev, oof


def run_tree_model(model_dict, model_name:str, features:List[str], params, oof:pd.DataFrame) -> (float, float, pd.DataFrame):
    (
        model,
        feature_importance_lst,
        fold_scores,
        final_valid_predictions,
        final_test_predictions,
    ) = train_xgb_model(
        train,
        test,
        model_dict[model_name],
        features,
        TARGET,
        Config.calc_probability,
        ID,
        params,
        Config.N_FOLDS,
        Config.seed,
    )

    cv_score, std_dev = show_fold_scores(fold_scores)
    show_tree_model_fi(model, features)

    oof = save_oof_predictions(model_name, final_valid_predictions, oof)
    oof.head()
    save_test_predictions(model_name, final_test_predictions, sample_submission, TARGET)

    return cv_score, std_dev, oof

In [24]:
%%time

def run_models4features(model_dict, model_lst:List[str], target:str, feature_lst:List[str], all_cv_scores:pd.DataFrame, linear_models:bool=True) -> pd.DataFrame:

    oof = train[[ID, target, "fold"]].copy().reset_index(drop=True).copy()
    oof.set_index(ID, inplace=True)

    for idx, m in enumerate(model_lst):
        model = model_lst[idx]
        start_time = time.time()

        print(f"Model={model}")

        params = {}
        if linear_models:
                cv_score, std_dev, oof = run_linear_model(model_dict, model, feature_lst, oof)

        else:
            cv_score, std_dev, oof = run_tree_model(model_dict, model, feature_lst, params, oof)

        run_time = time.time() - start_time

        score_dict = {"Model": model, "Score": cv_score, "StdDev": std_dev, "RunTime": run_time}
        all_cv_scores = all_cv_scores.append(score_dict, ignore_index=True)
        print(f"Model Run Time: {run_time:.2f}")

    return all_cv_scores




CPU times: user 11 µs, sys: 0 ns, total: 11 µs
Wall time: 14.8 µs


In [25]:
lgbm_params = {'n_estimators': Config.N_ESTIMATORS,
                 'num_rounds': 404,
                 'learning_rate': 0.19,
                 'num_leaves': 17,
                 'max_depth': 8,
                 'min_data_in_leaf': 36,
                 'lambda_l1': 0.96,
                 'lambda_l2': 0.01,
                 'min_gain_to_split': 11.32,
                 'bagging_fraction': 0.6,
                 'feature_fraction': 0.9}


lgbm_params = gpu_ify_lgbm(lgbm_params)
# if Config.gpu:
#     lgbm_params["device"] = "gpu"
#     lgbm_params["boosting_type"] = "gbdt"
#     lgbm_params["gpu_platform_id"] = 0
#     lgbm_params["gpu_device_id"] = 0

In [26]:
xgb_params = {
    "n_estimators": Config.N_ESTIMATORS,  # 10_000,
    "max_depth": 10,  # 10
    "objective": "reg:squarederror",
    #     "enable_categorical": True,  # Only works with gpu_hist
    #     "eval_metric": "mae",
    #     "metric": "mae",
    #     "enable_categorical": True,
    "n_jobs": 8,  # 4
    "seed": Config.seed,
    "tree_method": "hist",
    #         "gpu_id": 0,
    "subsample": 0.9,  # 0.7
    "colsample_bytree": 0.7,
    "use_label_encoder": False,
    "learning_rate": 0.05,  # 0.01
}

if Config.gpu:
    xgb_params["tree_method"] = "gpu_hist"
else:
    xgb_params["tree_method"] = "hist"

In [27]:
cb_params = {
    #     "learning_rate": 0.3277295792305584,
    "learning_rate": 0.05,
    "l2_leaf_reg": 3.1572972266001518,
    "bagging_temperature": 0.6799604234141348,
    "random_strength": 1.99590400593318,
    "depth": 10,
    "min_data_in_leaf": 93,
    # "iterations": 100,  # 10000
    "n_estimators": Config.N_ESTIMATORS,  # 10000
    "use_best_model": True,
    #     "task_type": "GPU",
    "random_seed": Config.seed,
}

cb_params = gpu_ify_cb(cb_params)
# if Config.gpu:
#     cb_params["task_type"] = "GPU"

In [28]:
lgbm_params = {
    "n_estimators": Config.GPU_N_ESTIMATORS,
    'max_depth': 9,
    'learning_rate': 0.01,
    'min_data_in_leaf': 36, 
    'num_leaves': 100, 
    'feature_fraction': 0.8, 
    'bagging_fraction': 0.89, 
    'bagging_freq': 5, 
    'lambda_l2': 28,
    
    'seed': Config.seed,
    'objective': 'regression',
#     'boosting_type': 'gbdt',
#     'device': 'gpu', 
#     'gpu_platform_id': 0,
#     'gpu_device_id': 0,
    'n_jobs': -1,
    'metric': 'rmse',
    'verbose': -1
}

if Config.gpu:
    lgbm_params["device"] = "gpu"
    lgbm_params["boosting_type"] = "gbdt"
    lgbm_params["gpu_platform_id"] = 0
    lgbm_params["gpu_device_id"] = 0

In [29]:
model_estimator_dict = {
    "xgb2": xgb.XGBRegressor(**xgb_params),
    "xgb_best_params": xgb.XGBRegressor(**best_xgb_params),

#     "lgbm1": lgb.LGBMRegressor(**lgbm_params),

    "cat1": cb.CatBoostRegressor(),
    "cat2": cb.CatBoostRegressor(**cb_params),
    "cat_best_params": cb.CatBoostRegressor(**best_cb_params),

    "xgb1": xgb.XGBRegressor(),
    "lgbm1": lgb.LGBMRegressor(),
    "lgbm1": lgb.LGBMRegressor(),
    "lgbm2": lgb.LGBMRegressor(
        learning_rate=0.05,
        max_depth=15,
        num_leaves=11,
        feature_fraction=0.3,
        subsample=0.1,
        n_jobs=-1,
    ),
    "lgbm3": lgb.LGBMRegressor(**lgbm_params),
    "lgbm_best_params": lgb.LGBMRegressor(**best_lgbm_params),


    "lin_reg": linear_model.LinearRegression(),
    "lasso": linear_model.Lasso(),
    "ridge": linear_model.Ridge(max_iter=7000),
    "ridge_25": linear_model.Ridge(fit_intercept=True, solver='auto', alpha=0.25, max_iter=7000),
    "ridge_50": linear_model.Ridge(fit_intercept=True, solver='auto', alpha=0.5, max_iter=7000),

}

## Tree Models

In [30]:
%%time

model_lst = ["xgb_best_params", "lgbm_best_params", "cat_best_params", "xgb1", "xgb2", "lgbm1", "lgbm2", "cat1", "cat2"]
# model_lst = ["lgbm1"]
# model_lst = = []
all_cv_scores = run_models4features(model_estimator_dict, model_lst, TARGET, FEATURES, all_cv_scores, linear_models=False)    

all_cv_scores.sort_values(by=["Score"], ascending=False)

Model=xgb_best_params
{}
========== Fold 1/5 ==========
fold: 1, Score: 1553391.0526323328, Run Time: 168.42
========== Fold 2/5 ==========
fold: 2, Score: 1580669.853882261, Run Time: 206.01
========== Fold 3/5 ==========
fold: 3, Score: 1565235.1726674687, Run Time: 182.47
========== Fold 4/5 ==========
fold: 4, Score: 1582651.659557184, Run Time: 176.16
========== Fold 5/5 ==========
fold: 5, Score: 1570138.1392809958, Run Time: 211.29
Scores -> Adjusted: 1559724.93646878 , mean: 1570417.17560405, std: 10692.23913526

=== Model Feature Importance ===
attic 0.096934184
hasPool 0.09449263
squareMeters 0.08531257
basement 0.07558604
made 0.07291393
hasGuestRoom 0.07136008
is_original 0.06956717
isNewBuilt 0.069519065
hasStorageRoom 0.060691174
garage 0.054430082
hasYard 0.052781004
hasStormProtector 0.050565645
numPrevOwners 0.045684114
numberOfRooms 0.040322904
floors 0.030530512
cityPartRange 0.029308869


,pred_xgb_best_params
id,
0,4934937.0
1,6801636.0
2,5106223.5
3,7831515.0
4,5337987.0


Mode
=== Target Value Counts ===
Model Run Time: 947.14
Model=lgbm_best_params
{}
========== Fold 1/5 ==========
[LightGBM] [Warning] bagging_fraction is set=0.8556709748326068, subsample=0.22 will be ignored. Current value: bagging_fraction=0.8556709748326068
[LightGBM] [Warning] feature_fraction is set=0.4151251951012078, colsample_bytree=0.5 will be ignored. Current value: feature_fraction=0.4151251951012078
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
fold: 1, Score: 398158.56215760467, Run Time: 216.52
========== Fold 2/5 ==========
[LightGBM] [Warning] bagging_fraction is set=0.8556709748326068, subsample=0.22 will be ignored. Current value: bagging_fraction=0.8556709748326068
[LightGBM] [Warning] feature_fraction is set=0.4151251951012078, colsample_bytree=0.5 will be ignored. Current value: feature_fraction=0.4151251951012078
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value

,pred_lgbm_best_params
id,
0,3.47402e+06
1,8.90182e+06
2,8.51374e+06
3,9.17626e+06
4,5.85675e+06


Mode
=== Target Value Counts ===
Model Run Time: 1096.76
Model=cat_best_params
{}
========== Fold 1/5 ==========
fold: 1, Score: 50448.25070081555, Run Time: 2.56
========== Fold 2/5 ==========
fold: 2, Score: 49976.05468304077, Run Time: 2.63
========== Fold 3/5 ==========
fold: 3, Score: 51220.50570192212, Run Time: 2.62
========== Fold 4/5 ==========
fold: 4, Score: 53042.29652157516, Run Time: 2.88
========== Fold 5/5 ==========
fold: 5, Score: 51315.14577593608, Run Time: 2.62
Scores -> Adjusted: 50154.41973647 , mean: 51200.45067666, std: 1046.03094018

=== Model Feature Importance ===
squareMeters 0.9999980029746096
garage 1.997025390334676e-06
is_original 0.0
hasGuestRoom 0.0
hasStorageRoom 0.0
attic 0.0
basement 0.0
hasStormProtector 0.0
isNewBuilt 0.0
made 0.0
numPrevOwners 0.0
cityPartRange 0.0
floors 0.0
hasPool 0.0
hasYard 0.0
numberOfRooms 0.0


...

,pred_cat1
id,
0,3.43265e+06
1,9.51099e+06
2,9.28103e+06
3,9.68102e+06
4,6.12538e+06


Mode
=== Target Value Counts ===
Model Run Time: 28.94
Model=cat2
{}
========== Fold 1/5 ==========
fold: 1, Score: 29719.48382289648, Run Time: 118.07
========== Fold 2/5 ==========
fold: 2, Score: 28814.88524723147, Run Time: 29.91
========== Fold 3/5 ==========
fold: 3, Score: 33430.968999055076, Run Time: 43.76
========== Fold 4/5 ==========
fold: 4, Score: 30758.985682419716, Run Time: 28.62
========== Fold 5/5 ==========
fold: 5, Score: 29323.46001391201, Run Time: 31.82
Scores -> Adjusted: 28769.44599318 , mean: 30409.55675310, std: 1640.11075993

=== Model Feature Importance ===
squareMeters 0.9956500359043124
garage 0.0011290328618585898
attic 0.0006914489379681839
hasGuestRoom 0.0005290232589550614
numPrevOwners 0.00034320451988401686
numberOfRooms 0.0003203043483924089
hasYard 0.0002998660921453093
cityPartRange 0.00024559957968358406
basement 0.00024417471537288034
hasStormProtector 0.00013837574627767373
hasStorageRoom 0.00011882039629183303
is_original 9.827314765083981e-

,pred_cat2
id,
0,3.47355e+06
1,9.50487e+06
2,9.26939e+06
3,9.68880e+06
4,6.16838e+06


Mode
=== Target Value Counts ===
Model Run Time: 252.92
CPU times: user 56min 30s, sys: 6min 52s, total: 1h 3min 22s
Wall time: 45min 34s


,Model,Score,StdDev,RunTime
0,xgb_best_params,1.57042e+06,10692.23914,947.14424
6,lgbm2,8.97806e+05,5866.33240,2.96038
1,lgbm_best_params,4.02359e+05,3657.86568,1096.76371
4,xgb2,1.45890e+05,1267.36845,372.36899
2,cat_best_params,5.12005e+04,1046.03094,13.83051
7,cat1,3.13229e+04,3065.54667,28.93580
8,cat2,3.04096e+04,1640.11076,252.92044
5,lgbm1,2.42096e+04,1081.72343,5.30491
3,xgb1,1.73011e+04,664.27167,14.54139


## Linear Models

In [31]:
model_lst = ["lin_reg", "lasso", "ridge", "ridge_25", "ridge_50"]
model_lst = ["lasso", "ridge",  "ridge_50"]
# model_lst = []
# all_cv_scores = run_models4features(model_lst, TARGET, FEATURES, all_cv_scores, linear_models=True)    
all_cv_scores = run_models4features(model_estimator_dict, model_lst, TARGET, FEATURES, all_cv_scores, linear_models=True)    

all_cv_scores.head()

Model=lasso
========== Fold 1/5 ==========
fold: 1, Score: 101625.26299581607, Run Time: 0.03
========== Fold 2/5 ==========
fold: 2, Score: 1556084.7606959804, Run Time: 0.04
========== Fold 3/5 ==========
fold: 3, Score: 1545878.3858512293, Run Time: 0.04
========== Fold 4/5 ==========
fold: 4, Score: 1543038.2759907155, Run Time: 0.04
========== Fold 5/5 ==========
fold: 5, Score: 1564073.158003011, Run Time: 0.04
Scores -> Adjusted: 681834.37298968 , mean: 1262139.96870735, std: 580305.59571767


,pred_lasso
id,
0,4.33578e+06
1,6.88117e+06
2,6.54651e+06
3,6.98478e+06
4,5.12189e+06


Mode
=== Target Value Counts ===
Model Run Time: 0.93
Model=ridge
========== Fold 1/5 ==========
fold: 1, Score: 101683.15383446707, Run Time: 0.04
========== Fold 2/5 ==========
fold: 2, Score: 1556120.0506844728, Run Time: 0.04
========== Fold 3/5 ==========
fold: 3, Score: 1545913.716666436, Run Time: 0.04
========== Fold 4/5 ==========
fold: 4, Score: 1543073.64435113, Run Time: 0.04
========== Fold 5/5 ==========
fold: 5, Score: 1564108.9079791966, Run Time: 0.04
Scores -> Adjusted: 681883.27912793 , mean: 1262179.89470314, std: 580296.61557521


,pred_ridge
id,
0,4.33580e+06
1,6.88111e+06
2,6.54645e+06
3,6.98471e+06
4,5.12188e+06


Mode
=== Target Value Counts ===
Model Run Time: 0.80
Model=ridge_50
========== Fold 1/5 ==========
fold: 1, Score: 101654.73414884658, Run Time: 0.02
========== Fold 2/5 ==========
fold: 2, Score: 1556102.1862014532, Run Time: 0.04
========== Fold 3/5 ==========
fold: 3, Score: 1545895.8076678624, Run Time: 0.04
========== Fold 4/5 ==========
fold: 4, Score: 1543055.7028033254, Run Time: 0.04
========== Fold 5/5 ==========
fold: 5, Score: 1564090.8047879785, Run Time: 0.04
Scores -> Adjusted: 681859.04647050 , mean: 1262159.84712189, std: 580300.80065140


,pred_ridge_50
id,
0,4.33580e+06
1,6.88115e+06
2,6.54648e+06
3,6.98475e+06
4,5.12188e+06


Mode
=== Target Value Counts ===
Model Run Time: 0.79


,Model,Score,StdDev,RunTime
0,xgb_best_params,1.57042e+06,10692.23914,947.14424
1,lgbm_best_params,4.02359e+05,3657.86568,1096.76371
2,cat_best_params,5.12005e+04,1046.03094,13.83051
3,xgb1,1.73011e+04,664.27167,14.54139
4,xgb2,1.45890e+05,1267.36845,372.36899


In [32]:
sample_submission.head(20)

,id,price,target_xgb_best_params,target_lgbm_best_params,target_cat_best_params,target_xgb1,target_xgb2,target_lgbm1,target_lgbm2,target_cat1,target_cat2,target_lasso,target_ridge,target_ridge_50
0,22730,4.63446e+06,4.29187e+06,4.20439e+06,4.75810e+06,4.73961e+06,4.68068e+06,4.75085e+06,4.32472e+06,4.75847e+06,4.71528e+06,4.76089e+06,4.76089e+06,4.76089e+06
1,22731,4.63446e+06,5.26752e+06,5.67761e+06,6.17728e+06,6.19620e+06,6.19695e+06,6.19086e+06,5.56337e+06,6.14545e+06,6.18016e+06,5.27998e+06,5.27995e+06,5.27996e+06
2,22732,4.63446e+06,5.62482e+06,8.33629e+06,9.03579e+06,9.05793e+06,8.96076e+06,9.03884e+06,7.31847e+06,9.04833e+06,9.06703e+06,6.51969e+06,6.51963e+06,6.51967e+06
3,22733,4.63446e+06,1.40921e+06,1.30834e+06,1.50304e+06,1.62152e+06,1.33877e+06,1.60190e+06,2.77953e+06,1.60946e+06,1.63177e+06,1.64894e+06,1.64907e+06,1.64901e+06
4,22734,4.63446e+06,3.65711e+06,5.87401e+06,6.73378e+06,6.76448e+06,6.56250e+06,6.74589e+06,6.03757e+06,6.68705e+06,6.70374e+06,5.35053e+06,5.35051e+06,5.35052e+06
5,22735,4.63446e+06,2.57636e+06,6.03520e+05,9.95024e+04,1.08245e+05,2.42312e+05,1.03186e+05,2.18509e+06,1.03126e+05,8.30254e+04,1.13674e+05,1.13860e+05,1.13769e+05
6,22736,4.63446e+06,5.87474e+06,8.60610e+06,9.86794e+06,9.91064e+06,9.47601e+06,9.86915e+06,7.40554e+06,9.89383e+06,9.83440e+06,6.44101e+06,6.44094e+06,6.44098e+06
7,22737,4.63446e+06,5.31543e+06,4.94998e+06,5.19533e+06,5.25057e+06,5.18810e+06,5.24446e+06,5.06112e+06,5.23341e+06,5.24837e+06,4.73045e+06,4.73045e+06,4.73045e+06
8,22738,4.63446e+06,4.10751e+06,5.64889e+06,5.63448e+06,5.55496e+06,5.42180e+06,5.56625e+06,5.13658e+06,5.58162e+06,5.53373e+06,4.83517e+06,4.83516e+06,4.83516e+06
9,22739,4.63446e+06,4.95655e+06,8.87103e+06,9.73308e+06,9.81226e+06,9.56863e+06,9.78990e+06,7.44687e+06,9.79073e+06,9.73978e+06,6.23997e+06,6.23991e+06,6.23994e+06


<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Blend Models</h1>
</div>

In [33]:
all_blend_scores = pd.DataFrame(
    {
        "Model": pd.Series(dtype="str"),
        "Score": pd.Series(dtype="float"),
        "StdDev": pd.Series(dtype="float"),
    }
)

In [34]:
sample_submission[TARGET] = (
#     (sample_submission["target_xgb_bp"] * 2 )
#     + (sample_submission["target_lgbm_bp"]  )
    (sample_submission["target_xgb1"] * 3 )
    + (sample_submission["target_lgbm1"])
#     + (sample_submission["target_lgbm2"])    
#     + (sample_submission["target_lgbm2"])
    + (sample_submission["target_cat1"] )
    + (sample_submission["target_cat2"] )    
#     + (sample_submission["target_cat_bp"] )
#     + (sample_submission["target_svc"] )
#     + (sample_submission["target_log_reg3"] )
#     + (sample_submission["target_cat2"] )
)/6

sample_submission[TARGET] = sample_submission[TARGET] #.astype(int)

In [35]:
sample_submission[[ID, TARGET]].to_csv("submission_wt_avg.csv", index=False)
sample_submission[[ID, TARGET]].tail(8)

,id,price
15146,37876,6.83323e+06
15147,37877,2.17543e+06
15148,37878,3.07695e+06
15149,37879,8.62007e+06
15150,37880,4.53579e+06
15151,37881,7.33784e+06
15152,37882,7.75234e+06
15153,37883,1.77830e+06


In [36]:
all_cv_scores.sort_values(by=["Score"], ascending=False)

,Model,Score,StdDev,RunTime
0,xgb_best_params,1.57042e+06,10692.23914,947.14424
10,ridge,1.26218e+06,580296.61558,0.79642
11,ridge_50,1.26216e+06,580300.80065,0.79171
9,lasso,1.26214e+06,580305.59572,0.92917
6,lgbm2,8.97806e+05,5866.33240,2.96038
1,lgbm_best_params,4.02359e+05,3657.86568,1096.76371
4,xgb2,1.45890e+05,1267.36845,372.36899
2,cat_best_params,5.12005e+04,1046.03094,13.83051
7,cat1,3.13229e+04,3065.54667,28.93580
8,cat2,3.04096e+04,1640.11076,252.92044


<div style="background-color:rgba(177, 156, 217, 0.6);border-radius:5px;display:fill"><h1 style="text-align: center;padding: 12px 0px 12px 0px;">Level 1 Stack Models</h1>
</div>

In [37]:
## TODO: Generate these dictionaries from model names

train_oof_dict = {
    "train_pred_cat1": "train_pred_cat1.csv",
    "train_pred_cat2": "train_pred_cat2.csv",
    "train_pred_lgbm1": "train_pred_lgbm1.csv",    
    "train_pred_lgbm2": "train_pred_lgbm2.csv",    
    "train_pred_xgb1": "train_pred_xgb1.csv"
}

test_pred_dict = {
    "submission_cat1": "submission_cat1.csv",
    "submission_cat2": "submission_cat2.csv",
    "submission_lgbm1": "submission_lgbm1.csv",
    "submission_lgbm2": "submission_lgbm2.csv",
    "submission_xgb1": "submission_xgb1.csv",
}

In [38]:
def blend_results(train_oof_dict, test_pred_dict):
    oof_df = pd.DataFrame()
    test_preds_df = pd.DataFrame()

    for name, train_oof_fname in train_oof_dict.items():
        fname = "../working/" + train_oof_fname
        print(f"Processing {name}, {train_oof_fname}")
        df = pd.read_csv(fname)
        print(df.head())
#         print(df.iloc[:,1])
        preds = pd.Series(df.iloc[:,1], name=name)
#         print(preds[:5])
        oof_df = pd.concat([oof_df, preds], axis=1)
    #     oof_df = pd.concat([oof_df, pd.Series(np.load(TRAIN_PATH / train_oof), name=name)], axis=1)

    for name, test_pred_fname in test_pred_dict.items():
        fname = "../working/" + test_pred_fname
        print(f"{name}, {test_pred_fname}")
        df = pd.read_csv(fname)
        print(df.head())
        preds = pd.Series(df.iloc[:,1], name=name)
        test_preds_df = pd.concat([test_preds_df, preds], axis=1)

    print("=== oof ===")
    print(oof_df.head())
    print("=== test_preds ===")
    print(test_preds_df.head())
    return oof_df, test_preds_df
    
# (oof_df, preds_df) = blend_results(train_oof_dict, test_pred_dict)    

In [39]:
def load_oof_results(train_oof_dict, test_pred_dict):
    oof_df = pd.DataFrame()
    test_preds_df = pd.DataFrame()

    for name, train_oof_fname in train_oof_dict.items():
        fname = "../working/" + train_oof_fname
        print(f"Processing {name}, {train_oof_fname}")
        df = pd.read_csv(fname)
        print(df.head())
#         print(df.iloc[:,1])
        preds = pd.Series(df.iloc[:,1], name=name)
#         print(preds[:5])
        oof_df = pd.concat([oof_df, preds], axis=1)
    #     oof_df = pd.concat([oof_df, pd.Series(np.load(TRAIN_PATH / train_oof), name=name)], axis=1)

    for name, test_pred_fname in test_pred_dict.items():
        fname = "../working/" + test_pred_fname
        print(f"{name}, {test_pred_fname}")
        df = pd.read_csv(fname)
        print(df.head())
        preds = pd.Series(df.iloc[:,1], name=name)
        test_preds_df = pd.concat([test_preds_df, preds], axis=1)

    print("=== oof ===")
    print(oof_df.head())
    print("=== test_preds ===")
    print(test_preds_df.head())
    return oof_df, test_preds_df
    
(oof_df, preds_df) = load_oof_results(train_oof_dict, test_pred_dict) 

Processing train_pred_cat1, train_pred_cat1.csv
   id    pred_cat1
0   0  3.43265e+06
1   1  9.51099e+06
2   2  9.28103e+06
3   3  9.68102e+06
4   4  6.12538e+06
Processing train_pred_cat2, train_pred_cat2.csv
   id    pred_cat2
0   0  3.47355e+06
1   1  9.50487e+06
2   2  9.26939e+06
3   3  9.68880e+06
4   4  6.16838e+06
Processing train_pred_lgbm1, train_pred_lgbm1.csv
   id   pred_lgbm1
0   0  3.45508e+06
1   1  9.52913e+06
2   2  9.29059e+06
3   3  9.74598e+06
4   4  6.17961e+06
Processing train_pred_lgbm2, train_pred_lgbm2.csv
   id   pred_lgbm2
0   0  3.81541e+06
1   1  7.60261e+06
2   2  7.71102e+06
3   3  7.88060e+06
4   4  5.62759e+06
Processing train_pred_xgb1, train_pred_xgb1.csv
   id  pred_xgb1
0   0  3429240.0
1   1  9518231.0
2   2  9268216.0
3   3  9764527.0
4   4  6179530.0
submission_cat1, submission_cat1.csv
      id        price
0  22730  4.75847e+06
1  22731  6.14545e+06
2  22732  9.04833e+06
3  22733  1.60946e+06
4  22734  6.68705e+06
submission_cat2, submission_c

In [40]:
oof_df.head()

,train_pred_cat1,train_pred_cat2,train_pred_lgbm1,train_pred_lgbm2,train_pred_xgb1
0,3.43265e+06,3.47355e+06,3.45508e+06,3.81541e+06,3429240.0
1,9.51099e+06,9.50487e+06,9.52913e+06,7.60261e+06,9518231.0
2,9.28103e+06,9.26939e+06,9.29059e+06,7.71102e+06,9268216.0
3,9.68102e+06,9.68880e+06,9.74598e+06,7.88060e+06,9764527.0
4,6.12538e+06,6.16838e+06,6.17961e+06,5.62759e+06,6179530.0


In [41]:
preds_df.head()

,submission_cat1,submission_cat2,submission_lgbm1,submission_lgbm2,submission_xgb1
0,4.75847e+06,4.71528e+06,4.75085e+06,4.32472e+06,4739607.5
1,6.14545e+06,6.18016e+06,6.19086e+06,5.56337e+06,6196195.5
2,9.04833e+06,9.06703e+06,9.03884e+06,7.31847e+06,9057930.0
3,1.60946e+06,1.63177e+06,1.60190e+06,2.77953e+06,1621524.5
4,6.68705e+06,6.70374e+06,6.74589e+06,6.03757e+06,6764482.0


In [42]:
type(preds_df)

pandas.core.frame.DataFrame

In [43]:
def run_lr(useful_features:List[str], TARGET:str, train_df:pd.DataFrame, test_df:pd.DataFrame) -> (List[float],List[float]):
    final_predictions = []
    scores = []

    kfold = model_selection.KFold(n_splits=Config.N_FOLDS, shuffle=True, random_state=Config.seed)

    for fold, (train_idx, valid_idx) in enumerate(kfold.split(train_df)):
        xtrain = train_df.iloc[train_idx].reset_index(drop=True)
        xvalid = train_df.iloc[valid_idx].reset_index(drop=True)

        xtest = test_df[useful_features].copy()

        ytrain = xtrain[TARGET]
        yvalid = xvalid[TARGET]

        xtrain = xtrain[useful_features]
        xvalid = xvalid[useful_features]

#         model = LogisticRegression()
        model = linear_model.LinearRegression()
        # Smaller C means more regularization; default=1.0
        # 2947.0517025518097
#         model = LogisticRegression(max_iter=500, C=2947.0517025518097, penalty='l2',solver='newton-cg')
#         model = LogisticRegression(C = 2947.0517025518097,
#                         max_iter = 500,
#                         penalty = 'l2',
#                         solver = 'liblinear')
        model.fit(xtrain, ytrain)

        preds_valid = model.predict_proba(xvalid)[:,-1]
        test_preds = model.predict_proba(xtest)[:,-1]

        final_predictions.append(test_preds)
#         score = metrics.roc_auc_score(yvalid, preds_valid)
        score = metrics.mean_squared_error(yvalid, preds_valid, squared=False)
        print(f"Fold={fold}, Score={score}")
        scores.append(score)
    return scores, final_predictions


In [44]:
# useful_features = ["pred_lda", "pred_gbc","pred_gbc2", "pred_cat_bp", "pred_cat1", "pred_lgbm1", "pred_lgbm2", "pred_lgbm_bp", "pred_xgb1", "pred_xgb_bp"]
useful_features = [ "train_pred_cat1", "train_pred_cat2", "train_pred_lgbm1", "train_pred_lgbm2", "train_pred_xgb1"]

In [45]:
oof_df[useful_features].head()

,train_pred_cat1,train_pred_cat2,train_pred_lgbm1,train_pred_lgbm2,train_pred_xgb1
0,3.43265e+06,3.47355e+06,3.45508e+06,3.81541e+06,3429240.0
1,9.51099e+06,9.50487e+06,9.52913e+06,7.60261e+06,9518231.0
2,9.28103e+06,9.26939e+06,9.29059e+06,7.71102e+06,9268216.0
3,9.68102e+06,9.68880e+06,9.74598e+06,7.88060e+06,9764527.0
4,6.12538e+06,6.16838e+06,6.17961e+06,5.62759e+06,6179530.0


In [46]:
# preds_df[useful_features].head()

In [47]:
# fold_scores, final_predictions = run_lr(useful_features, TARGET, oof_df, preds_df)
# test_preds = np.mean(np.column_stack(final_predictions), axis=1)
# cv_score, std_dev = show_fold_scores(fold_scores)
# create_submission("level1_lr", TARGET, test_preds)

In [48]:
pd.options.display.max_colwidth = 100
pd.set_option("display.max_rows", 999)
pd.set_option("display.precision", 5)
pd.options.display.float_format = '{:.2f}'.format
pd.options.display.max_colwidth

100

In [49]:
all_cv_scores.sort_values(by=["Score"], ascending=False)

,Model,Score,StdDev,RunTime
0,xgb_best_params,1570417.18,10692.24,947.14
10,ridge,1262179.89,580296.62,0.80
11,ridge_50,1262159.85,580300.80,0.79
9,lasso,1262139.97,580305.60,0.93
6,lgbm2,897806.46,5866.33,2.96
1,lgbm_best_params,402359.34,3657.87,1096.76
4,xgb2,145890.31,1267.37,372.37
2,cat_best_params,51200.45,1046.03,13.83
7,cat1,31322.89,3065.55,28.94
8,cat2,30409.56,1640.11,252.92
